In [0]:
import torch
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [80]:
my_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])
trainset = dsets.CIFAR10(root='./data',train=True,transform = my_transform,download=True)
validset = dsets.CIFAR10(root='./data',train=False,transform=my_transform,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [0]:
trainloader = torch.utils.data.DataLoader(dataset=trainset,batch_size=4,shuffle=True,num_workers=2)
validloader = torch.utils.data.DataLoader(dataset=validset,batch_size=4,shuffle=False,num_workers=2)

In [0]:
def imshow(img):
  img = img/2  + 0.5
  npimg = img.numpy()
  plt.imshow(np.transpose(npimg,(1,2,0)))
  plt.show()


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



In [0]:
class Net (nn.Module):
  def __init__(self):
    super(Net,self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.conv2 = nn.Conv2d(6,16,5)
    self.pool = nn.MaxPool2d(2,2)
    self.l1 = nn.Linear(16*5*5,120)
    self.l2 = nn.Linear(120,84)
    self.l3 = nn.Linear(84,10)

  def forward(self,x):
    out = self.pool(F.relu(self.conv1(x)))
    out = self.pool(F.relu(self.conv2(out)))
    out = out.view(-1,self.flat(out))
    out = F.relu(self.l1(out))
    out = F.relu(self.l2(out))
    out = self.l3(out)
    return out

  def flat(self,x1):
    dim = x1.shape[1:]
    n = 1
    for ns in dim:
      n *= ns
    return n



In [98]:
epochs = 3
model = Net()
optimizer = torch.optim.SGD(model.parameters(),lr = 0.001,momentum=0.9)
criterion = nn.CrossEntropyLoss()

for epoch in range(epochs):
  LOSS=0.0
  for i,data in enumerate(trainloader,0):
    x,y = data
    optimizer.zero_grad()
    z = model(x)
    loss = criterion(z,y)
    loss.backward()
    optimizer.step()
    LOSS +=loss.item()

    if (i%2000==1999):
      print('[%d, %5d] loss: %.3f' %
            (epoch+1,i+1,LOSS/2000))
      LOSS=0.0



[1,  2000] loss: 2.220
[1,  4000] loss: 1.853
[1,  6000] loss: 1.662
[1,  8000] loss: 1.601
[1, 10000] loss: 1.523
[1, 12000] loss: 1.471
[2,  2000] loss: 1.405
[2,  4000] loss: 1.367
[2,  6000] loss: 1.366
[2,  8000] loss: 1.306
[2, 10000] loss: 1.288
[2, 12000] loss: 1.280
[3,  2000] loss: 1.182
[3,  4000] loss: 1.213
[3,  6000] loss: 1.191
[3,  8000] loss: 1.204
[3, 10000] loss: 1.184
[3, 12000] loss: 1.193


In [99]:
correct = 0
total = 0
with torch.no_grad():
    for data in validloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 57 %
